На выбор: либо обучить сверточную нейронную сеть на любом датасете, который не использовался на уроке, либо реализовать нейронную сеть AlexNet на любом датасете.

In [1]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from skimage import transform

Using TensorFlow backend.


In [2]:
# установка параметров нейросети
batch_size = 64
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_alexnet_model.h5'

In [3]:
# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [4]:
"""

tmpFeatures = []
for feature in x_train:
    tmpFeature = skimage.transform.resize(feature, (224, 224), mode='constant')
    tmpFeatures.append(tmpFeature)

x_train_resized = np.asarray(tmpFeatures)
"""

"\n\ntmpFeatures = []\nfor feature in x_train:\n    tmpFeature = skimage.transform.resize(feature, (224, 224), mode='constant')\n    tmpFeatures.append(tmpFeature)\n\nx_train_resized = np.asarray(tmpFeatures)\n"

In [5]:
# x_train_resized.shape

<img src="AlexNet_Summary_Table.jpg">

Так как данные в датасете значительно меньше (32-32-3), чем входящая размерность Алекснет (227-227-3), то понизим показатели нашей копии, чтобы она могла работать.

In [6]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
# 1st convolutional layer
model.add(Conv2D(filters=96, kernel_size=(3,3), strides=(4,4), padding='valid', input_shape=x_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# 2nd convolutional layer
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# 3rd convolutional layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

# 4th convolutional layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

# 5th convolutional layer
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dropout(0.25))

#1st full connected layer
# model.add(Dense(9216))
# model.add(Activation('relu'))

#2nd full connected layer
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.25))

#3rd full connected layer
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 8, 96)          2688      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 256)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 384)         885120    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 384)         1327488   
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 2, 256)        

In [8]:
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)
    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Использование data augmentation в реальном времени
Epoch 1/10
782/782 [==============================] - 34s 43ms/step - loss: 2.1539 - accuracy: 0.1551 - val_loss: 1.8549 - val_accuracy: 0.2675
Epoch 2/10
782/782 [==============================] - 32s 41ms/step - loss: 1.7293 - accuracy: 0.3274 - val_loss: 1.5967 - val_accuracy: 0.3891
Epoch 3/10
782/782 [==============================] - 32s 41ms/step - loss: 1.5627 - accuracy: 0.4183 - val_loss: 1.4215 - val_accuracy: 0.4736
Epoch 4/10
782/782 [==============================] - 34s 44ms/step - loss: 1.4555 - accuracy: 0.4743 - val_loss: 1.3373 - val_accuracy: 0.5130
Epoch 5/10
782/782 [==============================] - 33s 43ms/step - loss: 1.3970 - accuracy: 0.4950 - val_loss: 1.3679 - val_accuracy: 0.5120
Epoch 6/10
782/782 [==============================] - 33s 42ms/step - loss: 1.3568 - accuracy: 0.5167 - val_loss: 1.2693 - val_accuracy: 0.5460
Epoch 7/10
782/782 [==============================] - 31s 40ms/step - loss: 1.3226 - 

In [10]:
# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

сохранить обученную модель как C:\Users\alibe\OneDrive\Geekbrains\Введение в нейронные сети\Homework\saved_models\keras_cifar10_alexnet_model.h5 


In [11]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 230us/step
Test loss: 1.2271061796188354
Test accuracy: 0.5615000128746033


Точность модели составила 56%, чтобы применить Алекснет пришлось адаптировать модель к маленьким данным - удалить один полносвязный слой, уменьшить размеры ядер у конволюционных слоев, увеличить размер батча, добавить дропауты. 